# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [25]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [26]:
# checking the current working directory
print(os.getcwd())

# Gets the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Creates a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# joins the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [27]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracts each data row one by one and appends it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            

# The below creates a smaller event data csv file called event_datafile_full csv that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [28]:
# checks the number of rows in the new csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [29]:
# This makes a connection to a Cassandra instance on a local machine 

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # Session is required to establish a connection and to execute queries
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [30]:
# Creates a Keyspace udacity
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [31]:
# Sets the KEYSPACE to udacity
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

**Define Pandas Row Factory**

In [32]:
def pandas_factory(colnames, rows):
    df = pd.DataFrame(rows, columns=colnames)
    return df
session.row_factory = pandas_factory

**PART 2 - Create Cassandra tables, populate and query them**

**Query 1** *To get the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession =4*

*Primary key set to sessionId, itemInSession as the Select query will be filtering on these and hence these should be the partition key.*

In [33]:
query = "CREATE TABLE IF NOT EXISTS artist_session"
query = query + "(sessionId int, \
                  itemInSession int, \
                  artist text, \
                  song text, \
                  length decimal , \
                   primary key(sessionId,itemInSession) \
                  )"
try:
    session.execute(query)
except Exception as e:
    print(e)  

                    

*Set file to the new csv file created. File is opened and each line read sequentially and cassandra table 'artist_session' is populated*

In [34]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Below code sets the insert query
        query = "INSERT INTO artist_session(sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES(%s, %s, %s, %s, %s)"
        ## Mapping column names.        
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))        



*Query the table 'artist_session' to get artist, song, length that was heard during sessionId = 338 and itemInSession =4*

In [35]:
query2 = "select sessionId, itemInSession, artist, song, length from artist_session WHERE sessionId = 338 AND itemInSession = 4" 
try: 
    rows = session.execute(query2) 
except Exception as e: 
    print(e)



*Display query results using Pandas Dataframe*

In [36]:
df = rows._current_rows
df.head()

,sessionid,iteminsession,artist,song,length
0,338,4,Faithless,Music Matters (Mark Knight Dub),495.3073


**Query 2** *Give the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182*

*Parition key chosen is userId and sessionId with clustering column of itemInSession*

In [38]:
## Query 2:
# Drop the table if it exists
query = "DROP TABLE IF EXISTS user_artist"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
# Create the table if it does not exist    
query1 = "CREATE TABLE IF NOT EXISTS user_artist "
query1 = query1 + "(userId int, \
                    sessionId int, \
                    artist text, \
                    song text, \
                    firstName text, \
                    lastName text, \
                    itemInSession int , \
                    primary key((userId,sessionId),itemInSession) \
                    )"
try:
    session.execute(query1)
except Exception as e:
    print(e)


*Set file to the new csv file created. File is opened and each line read sequentially and cassandra table 'user_artist' is populated*

In [39]:
# Set the file to the new file created
file = 'event_datafile_new.csv'

# Open the file and read each line sequentially and write to the table user_artist
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Query 2 variable assigned to the Insert Statement
        query2 = "INSERT INTO user_artist(userId, sessionId, artist, song, firstName, lastName, itemInSession)"
        query2 = query2 + " VALUES(%s, %s, %s, %s, %s, %s, %s)"
        ## Assign appropriate values to the table columns
        session.execute(query2, (int(line[10]), int(line[8]), line[0], line[9], line[1], line[4], int(line[3])))

*Query the table 'user_artist' to get user, artist, song that was heard by userId = 10 during sessionId = 182*

In [40]:
query3 = "select userid, sessionid, itemInSession, artist, firstName, lastName, song from user_artist WHERE userId = 10 AND sessionId = 182" 
try: 
    rows = session.execute(query3) 
except Exception as e: 
    print(e)

*Use Pandas Dataframe to display the results*

In [41]:
df = rows._current_rows
df.head()

,userid,sessionid,iteminsession,artist,firstname,lastname,song
0,10,182,0,Down To The Bone,Sylvie,Cruz,Keep On Keepin' On
1,10,182,1,Three Drives,Sylvie,Cruz,Greece 2000
2,10,182,2,Sebastien Tellier,Sylvie,Cruz,Kilometer
3,10,182,3,Lonnie Gordon,Sylvie,Cruz,Catch You Baby (Steve Pitron & Max Sanna Radio...


**Query 3** *Give every user name (first and last) in music app history who listened to the song 'All Hands Against His Own'*

*Partition key is song and userid as the query filter is on song and need to make it unique by combining with the userid*

In [42]:
## Query 3: 
#Create table if it does not exist

query = "CREATE TABLE IF NOT EXISTS user_song "
query = query + "(song text, userId int, firstName text, lastName text,  primary key(song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

*Set file to the new csv file created. File is opened and each line read sequentially and cassandra table 'user_song' is populated*

In [43]:
# Assign newly created file 
file = 'event_datafile_new.csv'

#Open file, read line by line sequentially and populate table user_song
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##  Prepare Insert statement
        query = "INSERT INTO user_song(song, userId, firstName, lastName)"
        query = query + " VALUES(%s, %s, %s, %s)"
        ## Assign correct values to table columns
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

*Query the table 'user_song' to get all users that listened to the song 'All Hands Against His Own'*

In [44]:
query = "select song, userId, firstName, lastName from user_song WHERE song = 'All Hands Against His Own'" 
try: 
    rows = session.execute(query) 
except Exception as e: 
    print(e)



*Display the query results using Pandas Dataframe*

In [45]:
df = rows._current_rows
df.head()

,song,userid,firstname,lastname
0,All Hands Against His Own,29,Jacqueline,Lynch
1,All Hands Against His Own,80,Tegan,Levine
2,All Hands Against His Own,95,Sara,Johnson


### Drop the tables before closing out the sessions

In [46]:
## Drop the table before closing out the sessions
query = "DROP TABLE IF EXISTS artist_session "
try:
    session.execute(query)
except Exception as e:
    print(e)
   
query = "DROP TABLE IF EXISTS user_artist"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS user_song "
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [47]:
session.shutdown()
cluster.shutdown()